<a href="https://colab.research.google.com/github/patrickbryant1/MoLPC/blob/master/MoLPC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MoLPC
**M**odelling **o**f **L**arge **P**rotein **C**omplexes

This directory contains a pipeline for predicting very large protein complexes using the
[FoldDock pipeline](https://gitlab.com/ElofssonLab/FoldDock) based on [AlphaFold2](https://www.nature.com/articles/s41586-021-03819-2).
\
AlphaFold2 is available under the [Apache License, Version 2.0](http://www.apache.org/licenses/LICENSE-2.0) and so is FoldDock, which is a derivative thereof.  \
The AlphaFold2 parameters are made available under the terms of the [CC BY 4.0 license](https://creativecommons.org/licenses/by/4.0/legalcode) and have not been modified.
\
MolPC is licensed under the [Apache License, Version 2.0](http://www.apache.org/licenses/LICENSE-2.0).
\
\
**You may not use these files except in compliance with the licenses.**

### MoLPC is available for local installation here: https://github.com/patrickbryant1/MoLPC

Please see the publication in *Nature Communications*: [Predicting the structure of large protein complexes using AlphaFold and Monte Carlo tree search](https://www.nature.com/articles/s41467-022-33729-4) for more information.

**DEBUGGING INFO**
If you are experiencing problems running MoLPC.
1. Try removing all files stored at your Google drive related to MoLPC after connecting.
2. Ensure the naming of the MSAs and chains are correct. Read the naming instructions carefully. If these are not accurate, MoLPC does not know what files to use.
3. Open a github issue at https://github.com/patrickbryant1/MoLPC.

If you like MoLPC, please star the [github repo](https://github.com/patrickbryant1/MoLPC).
\
If you use MoLPC in your research, please cite the publication in *Nature Communications*: [Predicting the structure of large protein complexes using AlphaFold and Monte Carlo tree search](https://www.nature.com/articles/s41467-022-33729-4).

In [ ]:
#@title Install dependencies
#@markdown Make sure your runtime is GPU.
#@markdown In the menu above do: Runtime --> Change runtime type --> Hardware accelerator (set to GPU)

#@markdown **Press play.**

#@markdown Simply press play on each cell below and follow the instructions.

#@markdown You will have to restart the runtime after this finishes to include the new packages.
#@markdown In the menu above do: Runtime --> Restart runtime

#@markdown Don't worry about all the errors that pip give below, these are resolved in the end.
!pip install --upgrade "jax[cuda12_pip]" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
!pip install ml-collections==0.1.1
!pip install dm-haiku==0.0.11
!pip install pandas==1.3.5
!pip install biopython==1.81
!pip install chex==0.1.5
!pip install dm-tree==0.1.8
!pip install immutabledict==2.0.0
!pip install scipy==1.7.3
!pip install tensorflow==2.11.0
!pip install py3Dmol
!pip install numpy --upgrade

In [2]:
#@title Check that the GPU is accessible. The response from this cell should be "gpu".
import jax
from jax.lib import xla_bridge
print(xla_bridge.get_backend().platform)

gpu


In [ ]:
#@title Clone MoLPC git
import shutil
try:
  shutil.rmtree('/content/MoLPC', ignore_errors=True)
except:
  print('')

!git clone https://github.com/patrickbryant1/MoLPC.git

In [4]:
#@title #Follow all steps outlined below to run the assembly pipeline
#@markdown To try the **test case** 1A8R, click the box "test_case". Then press the play button to the left.
\
#@markdown If you don't want to run the test case, **leave the box blank**.

#@markdown #Parameters
#@markdown - *ID* - name of output \
#@markdown - *SUBSIZE* - the size of the subcomponents to use for the assembly (2 or 3) \
##@markdown - *GET_ALL* - get all possible subcomponents or only ones according to specified interactions below \
#@markdown - *USEQs* and *STs* - Unique sequence in the complex and the stoichiometry of this. \
#@markdown Up to 5 unique sequences are allowed with a total of up to 50 chains. \
#@markdown (If more are required, please install the local version: https://github.com/patrickbryant1/MoLPC) \
##@markdown - *INTERACTIONS* - Interactions between chains (if known) \
#@markdown - **MSAs** - currently no MSA search is available directly in the browser, therefore you have to provide your own MSAs in a3m format and upload them here. \
#@markdown There are two ways of doing this: \
#@markdown 1. Search uniclust_30 locally with HHblits \
#@markdown 2. Go to https://toolkit.tuebingen.mpg.de/tools/hhblits \
#@markdown Paste each unique chain sequence in the search field in fasta format --> Submit. \
#@markdown When the search is finished, go to the tab "Query Template MSA" and "Download Full A3M" \
#@markdown - Upload the MSAs here: \
#@markdown Click the folder icon (Files) to the left and select the upload file icon. Upload your files.
#@markdown - **Make sure to name your MSAs according to the convention** ID_1, ..., ID_N (both here and for the uploaded files)
import sys, os
from google.colab import files
import pandas as pd
import numpy as np
import glob
sys.path.insert(0,'/content/MoLPC/src')
test_case = True #@param {type:"boolean"}
ID = "1A8R" #@param {type:"string"}
SUBSIZE = 3 #@param ["2", "3"] {type:"raw"}
GET_ALL = True
#It is 1=True, 0=False
if GET_ALL==True:
  GET_ALL=1
else:
  GET_ALL=0
INTERACTIONS = ""
#Check that get all is true if INTERACTIONS are empty
if INTERACTIONS=="":
  GET_ALL=1
USEQ1 = "PSLSKEAALVHEALVARGLETPLRPPVHEMDNETRKSLIAGHMTEIMQLLNLDLADDSLMETPHRIAKMYVDEIFSGLDYANFPKITLIENKMKVDEMVTVRDITLTSTCESHFVTIDGKATVAYIPKDSVIGLSKINRIVQFFAQRPQVQERLTQQILIALQTLLGTNNVAVSIDAVHYCVKARGIRDATSATTTTSLGGLFKSSQNTRHEFLRAVRHHN" #@param {type:"string"}
ST1 = 10 #@param ["0", 1","2", "3","4","5","6","7","8","9","10"] {type:"raw"}
MSA1 = "1A8R_1.a3m" #@param {type:"string"}
USEQ2 = "" #@param {type:"string"}
ST2 = 0 #@param ["0", 1","2", "3","4","5","6","7","8","9","10"] {type:"raw"}
MSA2 = "" #@param {type:"string"}
USEQ3 = "" #@param {type:"string"}
ST3 = 0 #@param ["0", 1","2", "3","4","5","6","7","8","9","10"] {type:"raw"}
MSA3 = "" #@param {type:"string"}
USEQ4 = "" #@param {type:"string"}
ST4 = 0 #@param ["0", 1","2", "3","4","5","6","7","8","9","10"] {type:"raw"}
MSA4 = "" #@param {type:"string"}
USEQ5 = "" #@param {type:"string"}
ST5 = 0 #@param ["0", 1","2", "3","4","5","6","7","8","9","10"] {type:"raw"}
MSA5 = "" #@param {type:"string"}

#Create DFs
USEQS, CHAINS = pd.DataFrame(), pd.DataFrame()
ALPHABET='ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz'
USEQS['Sequence']=[x for x in [USEQ1, USEQ2, USEQ3, USEQ4, USEQ5] if len(x)>0]
USEQS['SeqID']=np.arange(1,len(USEQS)+1)
USEQS['Stoichiometry']=[ST1, ST2, ST3, ST4, ST5][:len(USEQS)]
chain_useqs = []
for ind,row in USEQS.iterrows():
  chain_useqs.extend([row.SeqID]*row.Stoichiometry)
CHAINS['Useq']=chain_useqs
CHAINS['Chain']=[x for x in ALPHABET[:len(CHAINS)]]

OUTDIR="/content/"


In [5]:
#@title Step 1: MSA PIPELINE
#@markdown Now, a default MSA is read in - no search is performed here
sys.path.insert(0,'/content/MoLPC/src/')
#Get MSA
if test_case==True:
  MSADIR='/content/MoLPC/data/test/'
else:
  MSADIR='/content/'
  msas = glob.glob(MSADIR+'*.a3m')
  msa_ids = [x.split('/')[-1] for x in msas]
  #See if all the MSAs are provided
  for msa in [MSA1, MSA2, MSA3, MSA4, MSA5][:len(USEQS)]:
    if msa not in msa_ids:
      print(msa,'is missing.')
    else:
      print(msa, 'is uploaded')


#@markdown Write the Paired and Block Diagonalized MSAs to predict sub-components
from preprocess import preprocess_colab
preprocess_colab.create_folder_structure(MSADIR, ID, OUTDIR, USEQS, INTERACTIONS, CHAINS, GET_ALL, SUBSIZE)

Creating all interactions of size 3 ...


In [6]:
#@title Step 2: FOLDING PIPELINE
#Create structure dir
STRUCTURE_DIR=OUTDIR+"AF/"
if not os.path.exists(STRUCTURE_DIR):
  os.mkdir(STRUCTURE_DIR)
#Get the sub_ids and lengths
import glob
files = glob.glob(OUTDIR+'*.fasta')
sub_ids = {}
for filename in files:
  with open(filename, 'r') as file:
    for line in file:
      line = line.rstrip()[1:].split('|')
      sub_ids[line[0]]=line[-1].split('-')[:-1]
      break

#@markdown Get the AF2 params
import shutil
PARAMS=STRUCTURE_DIR+'params/'
if not os.path.exists(PARAMS):
  os.mkdir(PARAMS)
  !wget https://storage.googleapis.com/alphafold/alphafold_params_2021-07-14.tar
  shutil.move('/content/alphafold_params_2021-07-14.tar', PARAMS)
  #Extract
  !tar -xvf /content/AF/params/alphafold_params_2021-07-14.tar -C /content/AF/params/

In [ ]:
#@markdown Predict the subcomponents
sys.path.insert(0,'/content/MoLPC/src/AF2')
from AF2 import run_alphafold_colab
import collections
collections.Iterable = collections.abc.Iterable

##### AF2 CONFIGURATION #####
PARAM=STRUCTURE_DIR
PRESET='full_dbs' #Choose preset model configuration - no ensembling (full_dbs) and (reduced_dbs) or 8 model ensemblings (casp14).
MAX_RECYCLES=10 #max_recycles (default=3)
MODEL_NAME='model_1' #model_1_ptm

#Go through all subcomponents and predict their structure
for sub_id in sub_ids:
  #Check if predictions already exist
  if len(glob.glob(OUTDIR+sub_id+'/*.pdb'))>0:
    print('Prediction for',sub_id,'exists')
    continue
  else:
    print('Predicting subcomponent', sub_id)
  ####Get fasta file####
  FASTAFILE=OUTDIR+sub_id+'.fasta'
  ####Get chain break#### Note! This is now set for trimer subcomponents
  CB=np.cumsum([int(x) for x in sub_ids[sub_id]])
  CB = [str(x) for x in CB]
  ####Get MSAs####
  #HHblits paired
  PAIREDMSA=OUTDIR+sub_id+'_paired.a3m'
  ##HHblits block diagonalized
  BLOCKEDMSA=OUTDIR+sub_id+'_blocked.a3m'
  MSAS=[PAIREDMSA,BLOCKEDMSA] #Comma separated list of msa paths
  run_alphafold_colab.main([MODEL_NAME], 1, MAX_RECYCLES, STRUCTURE_DIR, FASTAFILE, sub_id, MSAS, CB, OUTDIR)

In [ ]:
#@title Step 3: ASSEMBLY PIPELINE
#@markdown Prepare the assembly
COMPLEXDIR=OUTDIR+'/assembly/complex/' #Where all the output for the complex assembly will be directed
PAIRDIR=OUTDIR+'/assembly/pairs/'
META=OUTDIR+'/assembly/meta.csv' #where to write all interactions
from complex_assembly import prepare_assembly_colab
#Make complex directory
if not os.path.exists(COMPLEXDIR):
  os.mkdir(OUTDIR+'/assembly')
  os.mkdir(COMPLEXDIR)
#Rewrite the FoldDock preds to have separate chains according to the fasta file seqlens
files = glob.glob(OUTDIR+ID+'*/*1.pdb')
if len(files)>0:
    for pdbname in files:
        chains = prepare_assembly_colab.read_all_chains_coords(pdbname)
        if len(chains.keys())>1:
            continue
        subid = pdbname.split('/')[-2]
        print(subid)
        #Rewrite the files
        prepare_assembly_colab.write_pdb(chains, pdbname.split('.')[0]+'_rw'+'.pdb')

#Copy the predictions to reflect all chains
prepare_assembly_colab.copy_uints(ID, OUTDIR, OUTDIR+'/assembly/', USEQS,INTERACTIONS, CHAINS, GET_ALL, SUBSIZE)
##Rewrite AF predicted complexes to have proper numbering and chain labels
files = glob.glob(OUTDIR+'/assembly/'+ID+'*/*.pdb')
if len(files)>0:
    for pdbname in files:
        chains = prepare_assembly_colab.read_all_chains_coords(pdbname)
        subid = pdbname.split('/')[-2]
        chain_names = subid.split('_')[-1]
        #Rewrite the files
        prepare_assembly_colab.write_pdb_chain_labels(chains, chain_names, OUTDIR+'/assembly/'+subid+'.pdb')
#Write all pairs
#It is necessary that the first unique chain is named A-..N for and the second N-... and so on
if not os.path.exists(PAIRDIR):
  os.mkdir(PAIRDIR)

prepare_assembly_colab.get_all_pairs(OUTDIR+'/assembly/', PAIRDIR, INTERACTIONS, GET_ALL, META)
#Cleanup
for filename in glob.glob(OUTDIR+'/assembly/'+ID+'_*.pdb'):
  os.remove(filename)
for dir in glob.glob(OUTDIR+'/assembly/'+ID+'_*'):
  if os.path.isdir(dir)==True:
    shutil.rmtree(dir)


In [ ]:
#@markdown Assemble: find the best non-overlapping path that connect all nodes using Monte Carlo tree search
META_DF=pd.read_csv(META)
CHAIN_SEQS=pd.read_csv(OUTDIR+'/assembly/'+ID+'_chains.csv')
from complex_assembly import mcts_colab
mcts_colab.assemble(META_DF, PAIRDIR, OUTDIR+'/assembly/plddt/', USEQS, CHAIN_SEQS, COMPLEXDIR)


In [ ]:
#@title Score the assembly and download the result
COMPLEXDIR=OUTDIR+'/assembly/complex/'
from google.colab import files
from complex_assembly import score_entire_complex_colab
score_entire_complex_colab.main(ID, COMPLEXDIR+'best_complex.pdb', COMPLEXDIR+'optimal_path.csv', USEQS, CHAINS, COMPLEXDIR+'scores.csv')

#Clean up files used in the assembly and scoring
#Pair dir
try:
  shutil.rmtree(PAIRDIR)
  shutil.rmtree(OUTDIR+'/assembly/plddt/')
  for subcomponent_dir in glob.glob(OUTDIR+ID+'*_*'):
    if os.path.isdir(subcomponent_dir)==True:
      shutil.rmtree(subcomponent_dir)

except:
  print('No dirs to remove')

#Download
files.download(COMPLEXDIR+'best_complex.pdb')


mpDockQ: 0.7677392355732878


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#@title Display 3D structure {run: "auto"}
#From: https://colab.research.google.com/github/sokrypton/ColabFold/blob/main/AlphaFold2.ipynb
import py3Dmol
import glob
import matplotlib.pyplot as plt

view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js',)
view.addModel(open(COMPLEXDIR+'best_complex.pdb','r').read(),'pdb')


for n,chain,color in zip(range(len(CHAINS)),list("ABCDEFGHIJKLMNOPQRSTUVWXYZ"),
                     ["lime","cyan","magenta","yellow","salmon","white","blue","orange",
                     "grey","brown","lime","cyan","magenta","yellow","salmon","white","blue","orange",
                     "grey","brown","lime","cyan","magenta","yellow","salmon","white","blue","orange",
                     "grey","brown"]):
      view.setStyle({'chain':chain},{'cartoon': {'color':color}})
#view.setStyle({'cartoon': {'color':'spectrum'}})
view.zoomTo()
view.show()


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol